# AI Projects Growth Rate

## Table of Contents
- [Introduction](#introduction)
- [Database Connection](#connect-to-the-augur-database)
- [Load the Repositories](#load-the-urls-of-ai-repositories)
- [Retrieve Repository IDs](#retrieve-the-repository-ids-and-the-repository-names)
- [Fetch the Contribution Data](#fetch-the-contribution-data)
  - [Pull Request Contribution Data](#pull-request-contribution-data)
  - [Commit Contribution Data](#commit-contribution-data)
  - [Issue Contribution Data](#issue-contribution-data)
  - [Pull Request Review Contribution Data](#pull-request-review-contribution-data)
  - [Message Contribution Data](#message-contribution-data)

## Introduction

In this notebook, we will try to perform growth rate analysis of various AI/ML projects by considering different types of contributions like pull requests, commits, messages, issues, reviews etc. We will first start by fetching the contribution data for various types and merge them into a singl dataset and in the later part, we will apply the formulas or the models described in this [document](https://docs.google.com/document/d/1ZkPCLNq5UBHrhTNIgAta9cFqevk2rVZ5Vxl9jNuRbQc/edit?usp=sharing).

In [1]:
# importing the required libraries

import json
import pandas as pd
from sqlalchemy import create_engine, text

## Connect to the Augur database

In [2]:
# Opening the JSON file containing database credentials and loading it into a dictionary
with open("data/il_ai_creds.json") as config_file:
    config = json.load(config_file)
    
# Creating a PostgreSQL database connection string using the credentials from the JSON file
database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(
    config['user'],        # Username
    config['password'],    # Password
    config['host'],        # Hostname
    config['port'],        # Port number
    config['database']     # Database name
)

# Assigning the connection string to a variable
connection_string = database_connection_string

# Creating a SQLAlchemy engine using the connection string
engine = create_engine(connection_string)

## Load the URLs of AI repositories

Retrieve the list of repositories that will used for the growth rate analysis

In [3]:
# Opening the JSON file containing AI repository data and loading it into a dictionary
f = open('ai_repos.json')
data = json.load(f)

# Closing the file after loading the data
f.close()  

# print the collected repositories
from pprint import pprint
pprint(data)


{'gen_ai': ['https://github.com/lucidrains/imagen-pytorch',
            'https://github.com/langchain-ai/langchain',
            'https://github.com/run-llama/llama_index',
            'https://github.com/microsoft/lora',
            'https://github.com/nvidia/nemo',
            'https://github.com/huggingface/peft',
            'https://github.com/microsoft/semantic-kernel',
            'https://github.com/chroma-core/chroma',
            'https://github.com/milvus-io/milvus',
            'https://github.com/qdrant/qdrant',
            'https://github.com/bigscience-workshop/promptsource',
            'https://github.com/automatic1111/stable-diffusion-webui'],
 'llm': ['https://github.com/huggingface/transformers',
         'https://github.com/huggingface/datasets',
         'https://github.com/huggingface/trl',
         'https://github.com/microsoft/deepspeed',
         'https://github.com/timdettmers/bitsandbytes',
         'https://github.com/mistralai/mistral-common',
         'ht

In [4]:
# Initializing an empty list to store repository git URLs
repo_git_set = []

# Extracting the list of repositories from the loaded JSON data
for key in data.keys():
    repo_git_set.extend(data.get(key))

## Retrieve the repository IDs and the repository names

Let's retrieve the repository IDs and names from the augur database.

In [5]:
# Initializing empty lists to store repository IDs and names
repo_set = []
repo_name_set = []

# Iterating through the list of repository git URLs
for repo_git in repo_git_set:
    # Creating a SQL query to fetch repository ID and name for each git URL
    repo_query = text(f"""
                    SET SCHEMA 'augur_data';
                    SELECT 
                        b.repo_id,
                        b.repo_name
                    FROM
                        repo_groups a,
                        repo b
                    WHERE
                        a.repo_group_id = b.repo_group_id AND
                        b.repo_git = '{repo_git}'
            """)

    # Using the connection to execute the query
    with engine.connect() as connection:
        t = connection.execute(repo_query)  # Executing the query
        results = t.mappings().all()  # Fetching all the results
        
        # Checking if results are found and extracting repo_id and repo_name
        if results:
            repo_id = results[0]['repo_id']
            repo_name = results[0]['repo_name']
        else:
            repo_id = None
            repo_name = None
        
        # Appending the fetched repository ID and name to the respective lists
        repo_set.append(repo_id)
        repo_name_set.append(repo_name)

# Printing the lists of repository IDs and names
print(repo_set)
print(repo_name_set)

[25495, 25498, 25497, 25501, 25500, 25504, 25503, 25557, 25502, 25499, 25511, 25514, 25515, 25505, 25512, 25516, 25507, 25506, 25510, 25509, 25508, 25513, 25518, 25519, 25523, 25522, 25520, 25521, 25517, 25511, 25533, 25525, 25530, 25524, 25528, 25532, 25529, 25481, 25527, 25543, 25541, 25537, 25546, 25534, 25540, 25538, 25545, 25535, 25542, 25536, 25539]
['numpy', 'tensorflow', 'networkx', 'pytorch', 'keras-io', 'tinygrad', 'pandas', 'polars', 'arrow', 'mlx', 'transformers', 'spacy', 'nltk', 'allennlp', 'gensim', 'corenlp', 'deepspeech', 'fasttext', 'sentence-transformers', 'opennmt', 'opennlp', 'cogcomp-nlp', 'mycroft-core', 'open-assistant', 'rhasspy', 'ovos-core', 'jarvis', 'leon', 'porcupine', 'transformers', 'datasets', 'trl', 'deepspeed', 'bitsandbytes', 'mistral-common', 'llama', 'text-to-text-transfer-transformer', 'instructlab', 'gemma', 'imagen-pytorch', 'langchain', 'llama_index', 'lora', 'nemo', 'peft', 'semantic-kernel', 'chroma', 'milvus', 'qdrant', 'promptsource', 'stab

Let's convert the data type of repo_set from a list to a tuple so that we can easily pass this in the sql queries.

In [6]:
repo_set_tuple = tuple(repo_set)

In [7]:
# Define a function to execute SQL queries and return a DataFrame
def execute_query(query, engine):
    with engine.connect() as connection:
        result = connection.execute(query)
        return pd.DataFrame(result.fetchall(), columns=result.keys())

## Fetch the contribution data

### Pull Request Contribution data

Let's get the count of pull requests for each repository in `repo_set`, grouped by the year and month of creation, and store the result in a pandas dataframe for further analysis.

In [8]:
# Query to fetch pull request data for repo_ids in repo_set
pull_requests_query = text(f"""
    SELECT 
        repo_id, 
        CAST(DATE_PART('year', pr_created_at) AS INTEGER) AS year,
        CAST(DATE_PART('month', pr_created_at) AS INTEGER) AS month,
        COUNT(pull_request_id) AS pull_request_count
    FROM 
        augur_data.pull_requests
    WHERE 
        repo_id IN :repo_set_tuple
        AND pr_created_at IS NOT NULL 
    GROUP BY 
        repo_id, year, month
    ORDER BY 
        repo_id, year, month;
""")

pull_requests_df = execute_query(pull_requests_query.bindparams(repo_set_tuple=repo_set_tuple), engine)

In [9]:
pull_requests_df

,repo_id,year,month,pull_request_count
0,25481,2024,2,93
1,25481,2024,3,390
2,25481,2024,4,183
3,25481,2024,5,120
4,25481,2024,6,180
...,...,...,...,...
2978,25557,2024,6,370
2979,25557,2024,7,301
2980,25557,2024,8,268
2981,25557,2024,9,258


### Commit contribution data

Let's get the count of commits for each repository in `repo_set`, grouped by the year and month of creation, and store the result in a pandas dataframe for further analysis.

In [10]:
# Query to fetch commits data for repo_ids in repo_set
commits_query = text(f"""
    SELECT 
        repo_id, 
        CAST(DATE_PART('year', cmt_author_timestamp) AS INTEGER) AS year,
        CAST(DATE_PART('month', cmt_author_timestamp) AS INTEGER) AS month,
        COUNT(cmt_id) AS commit_count
    FROM 
        augur_data.commits
    WHERE 
        repo_id IN :repo_set_tuple
        AND cmt_author_timestamp IS NOT NULL 
    GROUP BY 
        repo_id, year, month
    ORDER BY 
        repo_id, year, month;
""")

commits_df = execute_query(commits_query.bindparams(repo_set_tuple=repo_set_tuple), engine)

In [11]:
commits_df

,repo_id,year,month,commit_count
0,25481,2024,2,346
1,25481,2024,3,699
2,25481,2024,4,632
3,25481,2024,5,384
4,25481,2024,6,807
...,...,...,...,...
3503,25557,2024,5,1871
3504,25557,2024,6,3115
3505,25557,2024,7,1616
3506,25557,2024,8,2080


### Issue contribution data

Let's get the count of issues for each repository in `repo_set`, grouped by the year and month of creation, and store the result in a pandas dataframe for further analysis.

In [12]:
# Query to fetch issues data for repo_ids in repo_set
issues_query = text(f"""
    SELECT 
        repo_id, 
        CAST(DATE_PART('year', created_at) AS INTEGER) AS year,
        CAST(DATE_PART('month', created_at) AS INTEGER) AS month,
        
        COUNT(issue_id) AS issue_count
    FROM 
        augur_data.issues
    WHERE 
        repo_id IN :repo_set_tuple
        AND created_at IS NOT NULL 
    GROUP BY 
        repo_id, year, month
    ORDER BY 
        repo_id, year, month;
""")

issues_df = execute_query(issues_query.bindparams(repo_set_tuple=repo_set_tuple), engine)

In [13]:
issues_df

,repo_id,year,month,issue_count
0,25481,2024,2,63
1,25481,2024,3,216
2,25481,2024,4,93
3,25481,2024,5,60
4,25481,2024,6,107
...,...,...,...,...
2993,25557,2024,5,307
2994,25557,2024,6,305
2995,25557,2024,7,354
2996,25557,2024,8,264


### Pull Request Review contribution data

Let's get the count of pull request reviews for each repository in `repo_set`, grouped by the year and month of creation, and store the result in a pandas dataframe for further analysis.

In [14]:
# Query to fetch pull request reviews data for repo_ids in repo_set
pr_reviews_query = text(f"""
    SELECT 
        repo_id, 
        CAST(DATE_PART('year', pr_review_submitted_at) AS INTEGER) AS year,
        CAST(DATE_PART('month', pr_review_submitted_at) AS INTEGER) AS month,
        COUNT(pr_review_id) AS review_count
    FROM 
        augur_data.pull_request_reviews
    WHERE 
        repo_id IN :repo_set_tuple
        AND pr_review_submitted_at IS NOT NULL
    GROUP BY 
        repo_id, year, month
    ORDER BY 
        repo_id, year, month;
""")

pr_reviews_df = execute_query(pr_reviews_query.bindparams(repo_set_tuple=repo_set_tuple), engine)

In [15]:
pr_reviews_df

,repo_id,year,month,review_count
0,25481,2024,2,133
1,25481,2024,3,1173
2,25481,2024,4,614
3,25481,2024,5,581
4,25481,2024,6,1040
...,...,...,...,...
1867,25557,2024,4,374
1868,25557,2024,5,428
1869,25557,2024,6,387
1870,25557,2024,7,420


### Message contribution data

Let's get the count comments or messages for each repository in `repo_set`, grouped by the year and month of creation, and store the result in a pandas dataframe for further analysis.

In [16]:
# Query to fetch messages data for repo_ids in repo_set
messages_query = text(f"""
    SELECT 
        repo_id, 
        CAST(DATE_PART('year', msg_timestamp) AS INTEGER) AS year,
        CAST(DATE_PART('month', msg_timestamp) AS INTEGER) AS month,
        COUNT(msg_id) AS message_count
    FROM 
        augur_data.message
    WHERE 
        repo_id IN :repo_set_tuple
        AND msg_timestamp IS NOT NULL 
    GROUP BY 
        repo_id, year, month
    ORDER BY 
        repo_id, year, month;
""")

messages_df = execute_query(messages_query.bindparams(repo_set_tuple=repo_set_tuple), engine)

In [17]:
messages_df

,repo_id,year,month,message_count
0,25481,2024,2,297
1,25481,2024,3,2233
2,25481,2024,4,1307
3,25481,2024,5,1095
4,25481,2024,6,1733
...,...,...,...,...
3164,25557,2024,5,1775
3165,25557,2024,6,1764
3166,25557,2024,7,1769
3167,25557,2024,8,1555


In [18]:
# Merge all dataframes into one final dataframe

final_df = pull_requests_df.merge(commits_df, on=['repo_id', 'year', 'month'], how='outer')
final_df = final_df.merge(issues_df, on=['repo_id', 'year', 'month'], how='outer')
final_df = final_df.merge(pr_reviews_df, on=['repo_id', 'year', 'month'], how='outer')
final_df = final_df.merge(messages_df, on=['repo_id', 'year', 'month'], how='outer')

In [19]:
final_df

,repo_id,year,month,pull_request_count,commit_count,issue_count,review_count,message_count
0,25481,2024,2,93.0,346.0,63.0,133.0,297.0
1,25481,2024,3,390.0,699.0,216.0,1173.0,2233.0
2,25481,2024,4,183.0,632.0,93.0,614.0,1307.0
3,25481,2024,5,120.0,384.0,60.0,581.0,1095.0
4,25481,2024,6,180.0,807.0,107.0,1040.0,1733.0
...,...,...,...,...,...,...,...,...
3817,25557,2024,6,370.0,3115.0,305.0,387.0,1764.0
3818,25557,2024,7,301.0,1616.0,354.0,420.0,1769.0
3819,25557,2024,8,268.0,2080.0,264.0,281.0,1555.0
3820,25557,2024,9,258.0,1870.0,251.0,NaN,542.0


In [20]:
# Check for missing values in the final dataframe
missing_values = final_df.isnull().sum()
missing_values

repo_id                  0
year                     0
month                    0
pull_request_count     839
commit_count           314
issue_count            824
review_count          1950
message_count          653
dtype: int64

In [21]:
# Fill missing values with 0 and convert to integers
count_columns = ['pull_request_count', 'commit_count', 'issue_count', 'review_count', 'message_count']
final_df[count_columns] = final_df[count_columns].fillna(0).astype(int)

In [22]:
final_df['total_contributions'] = (
    final_df['pull_request_count'] +
    final_df['commit_count'] +
    final_df['issue_count'] +
    final_df['review_count'] +
    final_df['message_count']
)

In [23]:
# Print the final dataframe
final_df.head()

,repo_id,year,month,pull_request_count,commit_count,issue_count,review_count,message_count,total_contributions
0,25481,2024,2,93,346,63,133,297,932
1,25481,2024,3,390,699,216,1173,2233,4711
2,25481,2024,4,183,632,93,614,1307,2829
3,25481,2024,5,120,384,60,581,1095,2240
4,25481,2024,6,180,807,107,1040,1733,3867
